In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from IPython.display import display, Markdown

import sys
sys.path.append('.')
import time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


from util import (
  draw_graphs as _draw_graphs,
  plot_distance_heatmap,
  plot_max_match_results
)
from pipeline import make_pipeline

from mcopt import ot, MorseGraph

In [5]:
pipeline = make_pipeline()

In [6]:
datasets = {
  'heated_cylinder': dict(
    src_t = 800, 
    dest_t = 879, 
    m_pfgw = 0.95,
    m_pw = 0.95,
    m_pgw = 0.9667
  ),
  'red_sea': dict(
    src_t = 1,
    dest_t = 4,
    m_pfgw = 0.7375,
    m_pw = 0.8125,
    m_pgw = 0.85
  ),
  'navier_stokes': dict(
    src_t = 1, 
    dest_t = 4,
    m_pfgw = 0.8875,
    m_pw = 0.85,
    m_pgw = 0.9167
  ),
  'tangaroa': dict(
    src_t = 51, 
    dest_t = 52,
    m_pfgw = 0.925,
    m_pw = 0.925,
    m_pgw = 0.9167
  ),
  'wind': dict(
    src_t = 1, 
    dest_t = 10,
    m_pfgw = 0.8875,
    m_pw = 0.85,
    m_pgw = 0.75
  ),
}

for name, dataset in datasets.items():
  dataset['networks'] = pipeline.mm_network(name).build()
  dataset['attributes'] = pipeline.attributes(name).build()

> [mmnetwork:heated_cylinder] config unchanged, loading


> [attributes:heated_cylinder] config unchanged, loading
> [mmnetwork:red_sea] config unchanged, loading


> [attributes:red_sea] config unchanged, loading
> [mmnetwork:navier_stokes] config unchanged, loading


> [attributes:navier_stokes] config unchanged, loading
> [mmnetwork:tangaroa] config unchanged, loading


> [attributes:tangaroa] config unchanged, loading
> [mmnetwork:wind] config unchanged, loading


> [attributes:wind] config unchanged, loading


In [14]:
num_runs = 100

In [15]:
gw_runtimes = {}

for name, dataset in datasets.items():
  src_t = dataset['src_t']
  dest_t = dataset['dest_t']
  
  src_net = dataset['networks'].frames[src_t]
  dest_net = dataset['networks'].frames[dest_t]
  
  t0 = time.time()
  
  for _ in range(num_runs):
    ot.GW(src_net, dest_net)
  
  t1 = time.time()
  
  total = t1 - t0
  avg = total / num_runs
  
  gw_runtimes[name] = dict(total = total, avg = avg)

In [16]:
gw_runtimes

{'heated_cylinder': {'total': 1014.6101722717285, 'avg': 10.146101722717285},
 'red_sea': {'total': 53.09311056137085, 'avg': 0.5309311056137085},
 'navier_stokes': {'total': 27.753630876541138, 'avg': 0.27753630876541135},
 'tangaroa': {'total': 37.33500123023987, 'avg': 0.37335001230239867},
 'wind': {'total': 34.774120807647705, 'avg': 0.34774120807647707}}

In [17]:
fgw_runtimes = {}

for name, dataset in datasets.items():
  src_t = dataset['src_t']
  dest_t = dataset['dest_t']
  
  src_net = dataset['networks'].frames[src_t]
  dest_net = dataset['networks'].frames[dest_t]
  
  attributes = dataset['attributes']
  
  M = attributes.attrs[attributes.index_map[src_t], attributes.index_map[dest_t]]
  
  t0 = time.time()
  
  for _ in range(num_runs):
    ot.fGW(src_net, dest_net, M = M)
  
  t1 = time.time()
  
  total = t1 - t0
  avg = total / num_runs
  
  fgw_runtimes[name] = dict(total = total, avg = avg)

In [18]:
fgw_runtimes

{'heated_cylinder': {'total': 642.5448634624481, 'avg': 6.425448634624481},
 'red_sea': {'total': 24.97797155380249, 'avg': 0.24977971553802492},
 'navier_stokes': {'total': 13.031601667404175, 'avg': 0.13031601667404175},
 'tangaroa': {'total': 35.05556893348694, 'avg': 0.35055568933486936},
 'wind': {'total': 11.81725263595581, 'avg': 0.1181725263595581}}

In [19]:
w_runtimes = {}

for name, dataset in datasets.items():
  src_t = dataset['src_t']
  dest_t = dataset['dest_t']
  
  src_net = dataset['networks'].frames[src_t]
  dest_net = dataset['networks'].frames[dest_t]
  
  attributes = dataset['attributes']
  
  M = attributes.attrs[attributes.index_map[src_t], attributes.index_map[dest_t]]
  
  t0 = time.time()
  
  for _ in range(num_runs):
    ot.Wasserstein(src_net, dest_net, M = M)
  
  t1 = time.time()
  
  total = t1 - t0
  avg = total / num_runs
  
  w_runtimes[name] = dict(total = total, avg = avg)

In [20]:
w_runtimes

{'heated_cylinder': {'total': 166.22519850730896, 'avg': 1.6622519850730897},
 'red_sea': {'total': 18.340080499649048, 'avg': 0.18340080499649047},
 'navier_stokes': {'total': 18.366689920425415, 'avg': 0.18366689920425416},
 'tangaroa': {'total': 23.202397108078003, 'avg': 0.23202397108078002},
 'wind': {'total': 8.073045253753662, 'avg': 0.08073045253753662}}

In [21]:
pfgw_runtimes = {}

for name, dataset in datasets.items():
  src_t = dataset['src_t']
  dest_t = dataset['dest_t']
  
  src_net = dataset['networks'].frames[src_t]
  dest_net = dataset['networks'].frames[dest_t]
  
  attributes = dataset['attributes']
  
  M = attributes.attrs[attributes.index_map[src_t], attributes.index_map[dest_t]]
  m = dataset['m_pfgw']
  
  t0 = time.time()
  
  for _ in range(num_runs):
    ot.pfGW(src_net, dest_net, M = M, m=m)
  
  t1 = time.time()
  
  total = t1 - t0
  avg = total / num_runs
  
  pfgw_runtimes[name] = dict(total = total, avg = avg)

In [22]:
pfgw_runtimes

{'heated_cylinder': {'total': 512.3025839328766, 'avg': 5.123025839328766},
 'red_sea': {'total': 28.605802059173584, 'avg': 0.28605802059173585},
 'navier_stokes': {'total': 17.832030296325684, 'avg': 0.17832030296325685},
 'tangaroa': {'total': 29.30567193031311, 'avg': 0.2930567193031311},
 'wind': {'total': 9.839390754699707, 'avg': 0.09839390754699708}}

In [23]:
dataset = datasets['heated_cylinder']
attributes = dataset['attributes']

M = attributes.attrs[attributes.index_map[800], attributes.index_map[879]]

M.sum()

120792.5610797684

In [28]:
pw_runtimes = {}

for name, dataset in datasets.items():
  src_t = dataset['src_t']
  dest_t = dataset['dest_t']
  
  src_net = dataset['networks'].frames[src_t]
  dest_net = dataset['networks'].frames[dest_t]
  
  attributes = dataset['attributes']
  
  M = attributes.attrs[attributes.index_map[src_t], attributes.index_map[dest_t]]
  m = dataset['m_pw']
  
  t0 = time.time()
  
  for _ in range(num_runs):
    ot.pWasserstein(src_net, dest_net, M = M, m=m)
  
  t1 = time.time()
  
  total = t1 - t0
  avg = total / num_runs
  
  pw_runtimes[name] = dict(total = total, avg = avg)

heated_cylinder
red_sea
navier_stokes
tangaroa
wind


In [25]:
pw_runtimes

{'heated_cylinder': {'total': 152.8492341041565, 'avg': 1.528492341041565},
 'red_sea': {'total': 23.058550357818604, 'avg': 0.23058550357818602},
 'navier_stokes': {'total': 13.24488115310669, 'avg': 0.13244881153106688},
 'tangaroa': {'total': 21.563406467437744, 'avg': 0.21563406467437743},
 'wind': {'total': 13.741454124450684, 'avg': 0.13741454124450683}}

In [26]:
pgw_runtimes = {}

for name, dataset in datasets.items():
  src_t = dataset['dest_t']
  dest_t = dataset['dest_t']
  
  src_net = dataset['networks'].frames[src_t]
  dest_net = dataset['networks'].frames[dest_t]
  
  
  m = dataset['m_pgw']
  
  t0 = time.time()
  
  for _ in range(num_runs):
    ot.pGW(src_net, dest_net, m=m)
  
  t1 = time.time()
  
  total = t1 - t0
  avg = total / num_runs
  
  pgw_runtimes[name] = dict(total = total, avg = avg)

In [27]:
pgw_runtimes

{'heated_cylinder': {'total': 177.58443665504456, 'avg': 1.7758443665504455},
 'red_sea': {'total': 28.284102201461792, 'avg': 0.2828410220146179},
 'navier_stokes': {'total': 14.583408117294312, 'avg': 0.1458340811729431},
 'tangaroa': {'total': 25.238882064819336, 'avg': 0.25238882064819335},
 'wind': {'total': 12.868221998214722, 'avg': 0.1286822199821472}}

In [29]:
import pandas as pd

In [47]:
data = pd.concat([
  pd.DataFrame.from_dict(gw_runtimes, orient='index').rename(columns={'total': 'gw total (s)', 'avg': 'gw average (s)'}),
  pd.DataFrame.from_dict(fgw_runtimes, orient='index').rename(columns={'total': 'fgw total (s)', 'avg': 'fgw average (s)'}),
  pd.DataFrame.from_dict(w_runtimes, orient='index').rename(columns={'total': 'wasserstein total (s)', 'avg': 'wasserstein average (s)'}),
  pd.DataFrame.from_dict(pfgw_runtimes, orient='index').rename(columns={'total': 'pfGW total (s)', 'avg': 'pfGW average (s)'}),
  pd.DataFrame.from_dict(pgw_runtimes, orient='index').rename(columns={'total': 'pGW total (s)', 'avg': 'pGW average (s)'}),
  pd.DataFrame.from_dict(pw_runtimes, orient='index').rename(columns={'total': 'pW total (s)', 'avg': 'pW average (s)'}),
], axis=1)

data

,gw total (s),gw average (s),fgw total (s),fgw average (s),wasserstein total (s),wasserstein average (s),pfGW total (s),pfGW average (s),pGW total (s),pGW average (s),pW total (s),pW average (s)
heated_cylinder,1014.610172,10.146102,642.544863,6.425449,166.225199,1.662252,512.302584,5.123026,177.584437,1.775844,145.103498,1.451035
red_sea,53.093111,0.530931,24.977972,0.249780,18.340080,0.183401,28.605802,0.286058,28.284102,0.282841,16.576731,0.165767
navier_stokes,27.753631,0.277536,13.031602,0.130316,18.366690,0.183667,17.832030,0.178320,14.583408,0.145834,11.762072,0.117621
tangaroa,37.335001,0.373350,35.055569,0.350556,23.202397,0.232024,29.305672,0.293057,25.238882,0.252389,20.935448,0.209354
wind,34.774121,0.347741,11.817253,0.118173,8.073045,0.080730,9.839391,0.098394,12.868222,0.128682,7.915257,0.079153


In [49]:
data.to_csv('timing.csv')

In [11]:
for name, dataset in datasets.items():
  src_t = dataset['src_t']
  src_net = dataset['networks'].frames[src_t]
  
  dest_t = dataset['dest_t']
  dest_net = dataset['networks'].frames[dest_t]
  
  print(f'{name}-{src_t}: {len(src_net.space)}')
  print(f'{name}-{dest_t}: {len(dest_net.space)}')

heated_cylinder-800: 601
heated_cylinder-879: 575
red_sea-1: 135
red_sea-4: 171
navier_stokes-1: 116
navier_stokes-4: 130
tangaroa-51: 189
tangaroa-52: 180
wind-1: 83
wind-10: 99
